In [21]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

In [22]:
engine = create_engine('postgresql://localhost:5432/wa_leg_raw')

In [23]:
con = engine.connect()

In [24]:
vote_df = pd.read_sql_query('select * from "vote_api"',con=engine)

In [25]:
vote_df['bill_unique'] = vote_df['biennium'] + ' ' + vote_df['bill_id']

In [26]:
vote_df.head()

,biennium,bill_id,motion,sequence_number,vote,vote_date,voter_id,voter_name,voting_agency,bill_unique
0,1991-92,SHB 1001,Final Passage,4,Yea,1991-02-22T00:00:00,7,Anderson,House,1991-92 SHB 1001
1,1991-92,SHB 1001,Final Passage,4,Yea,1991-02-22T00:00:00,11,Appelwick,House,1991-92 SHB 1001
2,1991-92,SHB 1001,Final Passage,4,Yea,1991-02-22T00:00:00,17,Ballard,House,1991-92 SHB 1001
3,1991-92,SHB 1001,Final Passage,4,Yea,1991-02-22T00:00:00,23,Basich,House,1991-92 SHB 1001
4,1991-92,SHB 1001,Final Passage,4,Yea,1991-02-22T00:00:00,32,Beck,House,1991-92 SHB 1001


In [27]:
len(vote_df)

3042939

In [28]:
def change_biennium_to_year(biennium):
    return int(biennium[0:4])

In [29]:
vote_df['year'] = vote_df['biennium'].apply(change_biennium_to_year)

In [30]:
vote_df = vote_df.drop(['bill_id', 'biennium', 'motion'], axis=1)

In [31]:
vote_df['sequence_number'] = vote_df['sequence_number'].apply(int)

In [32]:
vote_df['voter_id'] = vote_df['voter_id'].apply(int)

In [33]:
vote_df['vote_date'] = pd.to_datetime(vote_df['vote_date'])

In [34]:
def change_vote_to_int(vote):
    if vote == 'Nay':
        return 0
    if vote == 'Yea':
        return 1
    if vote == 'Excused':
        return 2
    if vote == 'Absent':
        return 3

In [35]:
def change_agency_to_int(agency):
    if agency == 'House':
        return 0
    if agency == 'Senate':
        return 1

In [36]:
vote_df['vote'] = vote_df['vote'].apply(change_vote_to_int)

In [37]:
vote_df['voting_agency'] = vote_df['voting_agency'].apply(change_agency_to_int)

In [38]:
vote_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3042939 entries, 0 to 3042938
Data columns (total 8 columns):
sequence_number    int64
vote               int64
vote_date          datetime64[ns]
voter_id           int64
voter_name         object
voting_agency      int64
bill_unique        object
year               int64
dtypes: datetime64[ns](1), int64(5), object(2)
memory usage: 185.7+ MB


In [39]:
len(vote_df)

3042939

In [40]:
con.close()

In [41]:
engine = create_engine('postgresql://localhost:5432/wa_leg_staging')
con = engine.connect()

In [53]:
vote_df[3000000:].to_sql('vote', con, if_exists='append', index=False)

0:300000
300000:600000
600000:900000
900000:1200000
1200000:1500000
1500000:1800000
1800000:2100000
2100000:2400000
2400000:2700000
2700000:3000000
3000000: